#드라이브 마운트

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#라이브러리 추가하기

In [0]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets


SyntaxError: ignored

#트레이닝 파라메터 설정하기

In [0]:
lr = 1e-3
batch_size = 64
num_epoch = 10

num_freq = 100

ckpt_dir = './drive/My Drive/YouTube/001-pytorch-mnist/checkpoint'
log_dir = './drive/My Drive/YouTube/001-pytorch-mnist/mnist/log'
data_dir = './drive/My Drive/YouTube/001-pytorch-mnist/mnist/data'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


#네트워크 구축하기

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)

        x = self.fc2(x)

        return x

#네트워크를 저장하거나 불러오는 함수 작성

In [0]:
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

    # print('model_epoch%d.pth is saved.' % epoch)

def load(ckpt_dir, net, optim):
    if not os.path.exists(ckpt_dir):
      epoch = 0
      return net, optim, epoch

    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])

    # print('%s is loaded.' % ckpt_lst[-1])

    return net, optim, epoch

#학습 데이터 불러오기

In [0]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(root=data_dir, train=False, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)

num_data = len(loader.dataset)
num_batch = round(num_data / batch_size)


Extracting ./drive/My Drive/YouTube/001-pytorch-mnist/mnist/data/MNIST/raw/train-images-idx3-ubyte.gz to ./drive/My Drive/YouTube/001-pytorch-mnist/mnist/data/MNIST/raw



Extracting ./drive/My Drive/YouTube/001-pytorch-mnist/mnist/data/MNIST/raw/train-labels-idx1-ubyte.gz to ./drive/My Drive/YouTube/001-pytorch-mnist/mnist/data/MNIST/raw



Extracting ./drive/My Drive/YouTube/001-pytorch-mnist/mnist/data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./drive/My Drive/YouTube/001-pytorch-mnist/mnist/data/MNIST/raw



Extracting ./drive/My Drive/YouTube/001-pytorch-mnist/mnist/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./drive/My Drive/YouTube/001-pytorch-mnist/mnist/data/MNIST/raw
Processing...
Done!


#네트워크 생성하기

In [0]:
net = Net().to(device)
params = net.parameters()

#손실함수 등을 설정하기

In [0]:
fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

writer = SummaryWriter(log_dir=log_dir)

#네트워크 불러오기

In [0]:
st_epoch = 0
net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

#테스트 실행하기

In [0]:
with torch.no_grad():
    net.eval()

    loss_arr = []
    acc_arr = []

    for batch, (input, label) in enumerate(loader, 1):
        # forward propagation 하기
        input = input.to(device)
        label = label.to(device)

        output = net(input)
        pred = fn_pred(output)

        # backward propagation
        # optim.zero_grad()

        loss = fn_loss(output, label)
        # loss.backward()

        acc = fn_acc(pred, label)

        # optim.step()

        # 손실함수를 계산하기
        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        # print('TRAIN: EPOCH %d/%d | BATCH %04d/%04d | LOSS: %.4f | ACC: %.4f' %
        #       (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

        print('EVAL: BATCH %04d/%04d | LOSS: %.4f | ACC: %.4f' %
              (batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

    print('FINAL EVAL: LOSS: %.4f | ACC: %.4f' % (np.mean(loss_arr), np.mean(acc_arr)))


EVAL: BATCH 0001/0156 | LOSS: 0.0140 | ACC: 1.0000
EVAL: BATCH 0002/0156 | LOSS: 0.0145 | ACC: 0.9922
EVAL: BATCH 0003/0156 | LOSS: 0.0121 | ACC: 0.9948
EVAL: BATCH 0004/0156 | LOSS: 0.0114 | ACC: 0.9961
EVAL: BATCH 0005/0156 | LOSS: 0.0133 | ACC: 0.9938
EVAL: BATCH 0006/0156 | LOSS: 0.0206 | ACC: 0.9896
EVAL: BATCH 0007/0156 | LOSS: 0.0262 | ACC: 0.9866
EVAL: BATCH 0008/0156 | LOSS: 0.0325 | ACC: 0.9863
EVAL: BATCH 0009/0156 | LOSS: 0.0296 | ACC: 0.9878
EVAL: BATCH 0010/0156 | LOSS: 0.0318 | ACC: 0.9875
EVAL: BATCH 0011/0156 | LOSS: 0.0326 | ACC: 0.9872
EVAL: BATCH 0012/0156 | LOSS: 0.0344 | ACC: 0.9844
EVAL: BATCH 0013/0156 | LOSS: 0.0321 | ACC: 0.9856
EVAL: BATCH 0014/0156 | LOSS: 0.0326 | ACC: 0.9855
EVAL: BATCH 0015/0156 | LOSS: 0.0390 | ACC: 0.9844
EVAL: BATCH 0016/0156 | LOSS: 0.0436 | ACC: 0.9834
EVAL: BATCH 0017/0156 | LOSS: 0.0432 | ACC: 0.9835
EVAL: BATCH 0018/0156 | LOSS: 0.0429 | ACC: 0.9844
EVAL: BATCH 0019/0156 | LOSS: 0.0412 | ACC: 0.9852
EVAL: BATCH 0020/0156 | LOSS: 0